In [ ]:
import os
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [ ]:
root_dir = '/content/gdrive/My Drive/BILTZai/'

In [ ]:
os.chdir(root_dir)

In [ ]:
print('current working directory::',os.getcwd())

current working directory:: /content/gdrive/My Drive/BILTZai


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk 
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

In [ ]:
from nltk.tokenize import word_tokenize , sent_tokenize

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_parquet('classificationData.parquet')

In [ ]:
df.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

In [ ]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
5688397,US,46632376,R2O0G4Q1PNWG1F,B00DHZBM98,36942601,Rainbow Star Hard Cover Case for Motorola Droi...,Wireless,5,0,0,0,1,Awesome case,This case fit my phone perfectly and I loved t...,2014-01-21
8547800,US,21069192,R26FF3VKSSF9PN,B003NBTU2M,709773372,Retractable Car Charger for HTC EVO 4G,Wireless,5,1,1,0,1,Love this thing!,This was one of the most practical and economi...,2011-03-01
4478014,US,9715549,R1SJSBZJM2V11J,B006WCT7Q8,165672789,Micro USB Wall Charger for Sony Ericsson Vivaz...,Wireless,5,0,0,0,1,Five Stars,excelente,2014-08-07
4111190,US,6333661,R3GBSEQWBB0P6A,B00KR9FT4Q,897361851,RAVPower 2800mAh Galaxy S4 Replacement Battery...,Wireless,5,0,2,0,1,5 stars,I love it. And it's reliable.,2014-09-16
3221802,US,6464636,R2BGCP8L0OB7R6,B00D4BDG9O,198845889,Kyocera Duraxt E4277 Soft Touch Rubberized Hol...,Wireless,5,1,1,0,1,Five Stars,I got the right one this time,2014-12-12


In [ ]:
df['review_label'] = np.where(df.star_rating > 4 , 'positive' , (np.where(df.star_rating > 2 , 'moderate','negative')))

In [ ]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,review_label
5688397,US,46632376,R2O0G4Q1PNWG1F,B00DHZBM98,36942601,Rainbow Star Hard Cover Case for Motorola Droi...,Wireless,5,0,0,0,1,Awesome case,This case fit my phone perfectly and I loved t...,2014-01-21,positive
8547800,US,21069192,R26FF3VKSSF9PN,B003NBTU2M,709773372,Retractable Car Charger for HTC EVO 4G,Wireless,5,1,1,0,1,Love this thing!,This was one of the most practical and economi...,2011-03-01,positive
4478014,US,9715549,R1SJSBZJM2V11J,B006WCT7Q8,165672789,Micro USB Wall Charger for Sony Ericsson Vivaz...,Wireless,5,0,0,0,1,Five Stars,excelente,2014-08-07,positive
4111190,US,6333661,R3GBSEQWBB0P6A,B00KR9FT4Q,897361851,RAVPower 2800mAh Galaxy S4 Replacement Battery...,Wireless,5,0,2,0,1,5 stars,I love it. And it's reliable.,2014-09-16,positive
3221802,US,6464636,R2BGCP8L0OB7R6,B00D4BDG9O,198845889,Kyocera Duraxt E4277 Soft Touch Rubberized Hol...,Wireless,5,1,1,0,1,Five Stars,I got the right one this time,2014-12-12,positive


In [ ]:
df[df.review_label == 'negative']

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,review_label
8172510,US,11600646,R3HOCAJDWQZFB9,B006O45K9C,269348382,RND Power Solutions Deluxe Data Sync Cradle / ...,Wireless,1,1,4,0,1,"Wrong dock received, no 2nd battery charging slot",The dock I received was not what was advertise...,2012-03-13,negative
2629703,US,44539531,RQ83Q5SNQRDJF,B00KT4YLMU,942550196,docooler® Bluetooth Adapter Dongle Transmitter...,Wireless,1,0,1,0,1,One Star,It does not work.,2015-01-28,negative
3625725,US,22550885,R3P7X9TEMQNSYA,B00JPIAGW0,742272381,Huawei Ascend Mate2 4G LTE Smart Phone - 16GB ...,Wireless,1,4,14,0,1,One Star,no update,2014-11-04,negative
3980802,US,10112393,R31WT6V73U32XF,B00MA0OJWS,709164001,Gear Beast GearJelly Armor Apple iPhone 6 Flex...,Wireless,1,2,4,0,1,One Star,scratches the back of the phone!,2014-09-29,negative
1861961,US,11133132,RKV63UH97LIU6,B00F3583S8,543984775,Samsung Galaxy Note 3 Standard Battery,Wireless,1,0,0,0,1,Unacceptable battery - dangerous!,Didn't have it long and it began to expand and...,2015-03-26,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4484327,US,9280610,RXL3VT2ZJUL5A,B0098USUYG,47084439,BestDealUSA Great Fit Screw Type Short Top Roo...,Wireless,2,0,0,0,1,Two Stars,None,2014-08-06,negative
5137132,US,1908736,R2EYBLFCAERSOQ,B00BTNEAR6,336272712,Aria Rose Pink White Zebra Combo Hard Soft Hig...,Wireless,2,0,0,0,1,It's ok,None,2014-04-28,negative
7553805,US,17989523,R3GRR0VNSOJTW1,B003Y34SMG,968335071,Aimo Wireless IPHONE4GPCLP005H Rubber Essentia...,Wireless,2,0,0,0,1,"Fits great, but color is off",None,2013-01-07,negative
5958680,US,31485740,R1X5SQQCE967EE,B00EPLTC1I,325331105,"Nexus 5 Case, Ringke [Slim] Snug-Fit Slender [...",Wireless,2,0,0,0,1,OK Slim Case,None,2013-12-08,negative


In [ ]:
df.groupby(['review_label']).count()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
review_label,,,,,,,,,,,,,,,
moderate,116580,116580,116580,116580,116580,116580,116580,116580,116580,116580,116580,116580,116580,88947,116580
negative,92900,92900,92900,92900,92900,92900,92900,92900,92900,92900,92900,92900,92900,70824,92900
positive,240620,240620,240620,240620,240620,240620,240620,240620,240620,240620,240620,240620,240620,183397,240620


In [ ]:
df.shape

(450100, 16)

In [ ]:
df = df[~df.review_body.isna()]

In [ ]:
df.shape

(343168, 16)

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
df['review_clean'] = df['review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,review_label,review_clean
5688397,US,46632376,R2O0G4Q1PNWG1F,B00DHZBM98,36942601,Rainbow Star Hard Cover Case for Motorola Droi...,Wireless,5,0,0,0,1,Awesome case,This case fit my phone perfectly and I loved t...,2014-01-21,positive,This case fit phone perfectly I loved stars lo...
8547800,US,21069192,R26FF3VKSSF9PN,B003NBTU2M,709773372,Retractable Car Charger for HTC EVO 4G,Wireless,5,1,1,0,1,Love this thing!,This was one of the most practical and economi...,2011-03-01,positive,This one practical economic purchases I made l...
4478014,US,9715549,R1SJSBZJM2V11J,B006WCT7Q8,165672789,Micro USB Wall Charger for Sony Ericsson Vivaz...,Wireless,5,0,0,0,1,Five Stars,excelente,2014-08-07,positive,excelente
4111190,US,6333661,R3GBSEQWBB0P6A,B00KR9FT4Q,897361851,RAVPower 2800mAh Galaxy S4 Replacement Battery...,Wireless,5,0,2,0,1,5 stars,I love it. And it's reliable.,2014-09-16,positive,I love it. And reliable.
3221802,US,6464636,R2BGCP8L0OB7R6,B00D4BDG9O,198845889,Kyocera Duraxt E4277 Soft Touch Rubberized Hol...,Wireless,5,1,1,0,1,Five Stars,I got the right one this time,2014-12-12,positive,I got right one time


In [ ]:
df['review_tokenize'] = df['review_clean'].apply(word_tokenize)

In [ ]:
df['review_sentence_tokenize'] = df['review_clean'].apply(sent_tokenize)

In [ ]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,review_label,review_clean,review_tokenize,review_sentence_tokenize
5688397,US,46632376,R2O0G4Q1PNWG1F,B00DHZBM98,36942601,Rainbow Star Hard Cover Case for Motorola Droi...,Wireless,5,0,0,0,1,Awesome case,This case fit my phone perfectly and I loved t...,2014-01-21,positive,This case fit phone perfectly I loved stars lo...,"[This, case, fit, phone, perfectly, I, loved, ...",[This case fit phone perfectly I loved stars l...
8547800,US,21069192,R26FF3VKSSF9PN,B003NBTU2M,709773372,Retractable Car Charger for HTC EVO 4G,Wireless,5,1,1,0,1,Love this thing!,This was one of the most practical and economi...,2011-03-01,positive,This one practical economic purchases I made l...,"[This, one, practical, economic, purchases, I,...",[This one practical economic purchases I made ...
4478014,US,9715549,R1SJSBZJM2V11J,B006WCT7Q8,165672789,Micro USB Wall Charger for Sony Ericsson Vivaz...,Wireless,5,0,0,0,1,Five Stars,excelente,2014-08-07,positive,excelente,[excelente],[excelente]
4111190,US,6333661,R3GBSEQWBB0P6A,B00KR9FT4Q,897361851,RAVPower 2800mAh Galaxy S4 Replacement Battery...,Wireless,5,0,2,0,1,5 stars,I love it. And it's reliable.,2014-09-16,positive,I love it. And reliable.,"[I, love, it, ., And, reliable, .]","[I love it., And reliable.]"
3221802,US,6464636,R2BGCP8L0OB7R6,B00D4BDG9O,198845889,Kyocera Duraxt E4277 Soft Touch Rubberized Hol...,Wireless,5,1,1,0,1,Five Stars,I got the right one this time,2014-12-12,positive,I got right one time,"[I, got, right, one, time]",[I got right one time]


In [ ]:
df['labels'] = np.where(df.review_label == 'positive' , 0 , (np.where(df.review_label == 'negative' , 1, 2)))

In [ ]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,review_label,review_clean,review_tokenize,review_sentence_tokenize,labels
5688397,US,46632376,R2O0G4Q1PNWG1F,B00DHZBM98,36942601,Rainbow Star Hard Cover Case for Motorola Droi...,Wireless,5,0,0,0,1,Awesome case,This case fit my phone perfectly and I loved t...,2014-01-21,positive,This case fit phone perfectly I loved stars lo...,"[This, case, fit, phone, perfectly, I, loved, ...",[This case fit phone perfectly I loved stars l...,0
8547800,US,21069192,R26FF3VKSSF9PN,B003NBTU2M,709773372,Retractable Car Charger for HTC EVO 4G,Wireless,5,1,1,0,1,Love this thing!,This was one of the most practical and economi...,2011-03-01,positive,This one practical economic purchases I made l...,"[This, one, practical, economic, purchases, I,...",[This one practical economic purchases I made ...,0
4478014,US,9715549,R1SJSBZJM2V11J,B006WCT7Q8,165672789,Micro USB Wall Charger for Sony Ericsson Vivaz...,Wireless,5,0,0,0,1,Five Stars,excelente,2014-08-07,positive,excelente,[excelente],[excelente],0
4111190,US,6333661,R3GBSEQWBB0P6A,B00KR9FT4Q,897361851,RAVPower 2800mAh Galaxy S4 Replacement Battery...,Wireless,5,0,2,0,1,5 stars,I love it. And it's reliable.,2014-09-16,positive,I love it. And reliable.,"[I, love, it, ., And, reliable, .]","[I love it., And reliable.]",0
3221802,US,6464636,R2BGCP8L0OB7R6,B00D4BDG9O,198845889,Kyocera Duraxt E4277 Soft Touch Rubberized Hol...,Wireless,5,1,1,0,1,Five Stars,I got the right one this time,2014-12-12,positive,I got right one time,"[I, got, right, one, time]",[I got right one time],0


In [ ]:
df.groupby(['labels']).count()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,review_label,review_clean,review_tokenize,review_sentence_tokenize
labels,,,,,,,,,,,,,,,,,,,
0,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397
1,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824
2,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947


In [ ]:
df.groupby(['review_label']).count()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,review_clean,review_tokenize,review_sentence_tokenize,labels
review_label,,,,,,,,,,,,,,,,,,,
moderate,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947,88947
negative,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824,70824
positive,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397,183397


In [ ]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,review_label,review_clean,review_tokenize,review_sentence_tokenize,labels
5688397,US,46632376,R2O0G4Q1PNWG1F,B00DHZBM98,36942601,Rainbow Star Hard Cover Case for Motorola Droi...,Wireless,5,0,0,0,1,Awesome case,This case fit my phone perfectly and I loved t...,2014-01-21,positive,This case fit phone perfectly I loved stars lo...,"[This, case, fit, phone, perfectly, I, loved, ...",[This case fit phone perfectly I loved stars l...,0
8547800,US,21069192,R26FF3VKSSF9PN,B003NBTU2M,709773372,Retractable Car Charger for HTC EVO 4G,Wireless,5,1,1,0,1,Love this thing!,This was one of the most practical and economi...,2011-03-01,positive,This one practical economic purchases I made l...,"[This, one, practical, economic, purchases, I,...",[This one practical economic purchases I made ...,0
4478014,US,9715549,R1SJSBZJM2V11J,B006WCT7Q8,165672789,Micro USB Wall Charger for Sony Ericsson Vivaz...,Wireless,5,0,0,0,1,Five Stars,excelente,2014-08-07,positive,excelente,[excelente],[excelente],0
4111190,US,6333661,R3GBSEQWBB0P6A,B00KR9FT4Q,897361851,RAVPower 2800mAh Galaxy S4 Replacement Battery...,Wireless,5,0,2,0,1,5 stars,I love it. And it's reliable.,2014-09-16,positive,I love it. And reliable.,"[I, love, it, ., And, reliable, .]","[I love it., And reliable.]",0
3221802,US,6464636,R2BGCP8L0OB7R6,B00D4BDG9O,198845889,Kyocera Duraxt E4277 Soft Touch Rubberized Hol...,Wireless,5,1,1,0,1,Five Stars,I got the right one this time,2014-12-12,positive,I got right one time,"[I, got, right, one, time]",[I got right one time],0


In [ ]:
df.to_csv('labeled_sentiment_data.csv')

In [ ]:
train , test = train_test_split(
    df,
    test_size = 0.20,                 #showing train - test split in analysis , it is specifically code in neural file again
    stratify = df.labels.values
)

In [ ]:
train.to_csv('sentiment-train.csv')

In [ ]:
test.to_csv('sentiment-test.csv')